### 1. Import & 데이터 로드

In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os

# 현재 폴더의 상위 상위 폴더 경로를 가져옵니다.
parent_dir = os.path.dirname(os.getcwd())
grandparents_dir = os.path.dirname(parent_dir)

# 상위 상위 폴더 경로를 sys.path에 추가합니다.
sys.path.insert(0, parent_dir)
sys.path.insert(0, grandparents_dir)

# 패키지 경로를 직접 import합니다.
from socceraction.data.statsbomb import StatsBombLoader
import socceraction.spadl as spadl
import socceraction.vaep.formula as vaepformula

In [2]:
# -*- coding: utf-8 -*-
import math
from tqdm import tqdm
from collections import deque
import random
import numpy as np
import pandas as pd
import ast
import random as ran
import datetime
from time import time, sleep
import matplotlib.pyplot as plt
import plotly
import plotly.graph_objects as go
import plotly.express as px
from utils import handle_trace_length, get_together_training_batch, compromise_state_trace_length
import matplotlib.pyplot as plt
from skimage.transform import resize
from skimage.color import rgb2gray
from scipy.io import savemat

In [3]:
import csv
import tensorflow as tf
import scipy.io as sio
import torch
import torch.optim as optim
import torch.nn as nn
from nn.td_two_tower_lstm import TD_Prediction_TT_Embed
import matplotlib.pyplot as plt
from torch.utils.tensorboard import SummaryWriter
from utils import handle_trace_length, get_together_training_batch, compromise_state_trace_length
from configuration import MODEL_TYPE, MAX_TRACE_LENGTH, FEATURE_NUMBER, BATCH_SIZE, GAMMA, H_SIZE, \
    model_train_continue, FEATURE_TYPE, ITERATE_NUM, learning_rate, SPORT, save_mother_dir, hidden_dim, lr, gamma, batch_size, memory_size, max_trace_length, output_dim, num_layers

2024-08-23 09:49:06.689024: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-23 09:49:06.816364: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-23 09:49:06.849086: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-23 09:49:07.047660: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-23 09:49:09.261192: W tensorflow/compiler/tf2

In [4]:
gim_folder = os.path.dirname(os.getcwd())

LOG_DIR = str(gim_folder) + save_mother_dir + "/models/hybrid_sl_log_NN/Scale-three-cut_together_log_train_feature" + str(
    FEATURE_TYPE) + "_batch" + str(
    BATCH_SIZE) + "_iterate" + str(
    ITERATE_NUM) + "_lr" + str(
    learning_rate) + "_" + str(MODEL_TYPE) + "_MaxTL" + str(MAX_TRACE_LENGTH)
SAVED_NETWORK = str(gim_folder) + save_mother_dir + "/models/hybrid_sl_saved_NN/Scale-three-cut_together_saved_networks_feature" + str(
    FEATURE_TYPE) + "_batch" + str(
    BATCH_SIZE) + "_iterate" + str(
    ITERATE_NUM) + "_lr" + str(
    learning_rate) + "_" + str(MODEL_TYPE) + "_MaxTL" + str(MAX_TRACE_LENGTH)

In [17]:
# GIM 계산하기 위한 각종 파일들을 저장할 경로 설정

if learning_rate == 1e-6:
    learning_rate_write = 6
elif learning_rate == 1e-5:
    learning_rate_write = 5
elif learning_rate == 1e-4:
    learning_rate_write = 4
else:
    raise ValueError("undefined learning_rate")

IS_POSSIBILITY = True
if_correct_velocity = "_v_correct_"

model_data_store_dir = "../LOG_NETWORK/models/TWOTOWER-LSTM-SOCCER-Training-All-feature{0}-scale" \
                            "-neg_reward{1}_length-dynamic".format(str(FEATURE_TYPE),
                                                                   if_correct_velocity)

player_game_info_dir = "../datastore"
player_info_dir = "../player_database.csv"
data_name = "model_three_cut_together_predict_Feature{0}_Iter{1}_lr{2}_Batch{3}_MaxLength{4}_Type{5}{6}".format(
    str(FEATURE_TYPE),
    str(ITERATE_NUM),
    str(learning_rate_write),
    str(BATCH_SIZE),
    str(MAX_TRACE_LENGTH),
    str(MODEL_TYPE),
    if_correct_velocity)

### 2. Q_value prediction

In [61]:
def game_predict(FEATURE_NUMBER, hidden_dim, MAX_TRACE_LENGTH, learning_rate, SAVED_NETWORK, SPORT, MODEL_VERSION, GAME_ID):
    ## loading network
    # 모델 불러오기
    model = TD_Prediction_TT_Embed(FEATURE_NUMBER, hidden_dim, MAX_TRACE_LENGTH, learning_rate)
    check_path = os.path.join(SAVED_NETWORK, f"{SPORT}-game-{MODEL_VERSION}.pt")
    checkpoint = torch.load(check_path)  # Load checkpoint
    model.load_state_dict(checkpoint['model_state_dict'])
    model.optimizer_home.load_state_dict(checkpoint['home_optimizer_state_dict'])
    model.optimizer_away.load_state_dict(checkpoint['away_optimizer_state_dict'])

    ## load data
    game_id = GAME_ID
    DATA_STORE = "../datastore"
    game_list = []
    trace_length_list = []
    reward_list = []

    dir_game = f'{game_id}'
    state = sio.loadmat(DATA_STORE + "/" + dir_game + "/" + 'rnn_input')
    reward = sio.loadmat(DATA_STORE + "/" + dir_game + "/" + 'rnn_reward')
    trace_length = sio.loadmat(DATA_STORE + "/" + dir_game + "/" + 'trace')
    game_list += (state['data'])[0].tolist()
    reward_list += (reward['data'])[0].tolist()
    trace_length_list += (trace_length['data'])[0].tolist()
    home_away_indicator = [1 if state[-1][10] > 1 else 0 for state in game_list]
    state_trace_length, state_input, reward = compromise_state_trace_length(trace_length_list, game_list, reward_list, MAX_TRACE_LENGTH)


    ## Prediction
    # get the batch variables
    y_batch = []
    # tensor 변환
    trace_batch_tensor = torch.tensor(state_trace_length, dtype=torch.int32)
    s_batch_tensor = torch.tensor(state_input, dtype=torch.float32)
    home_away_indicator_tensor = torch.tensor(home_away_indicator, dtype=torch.bool)
    # 모델을 평가 모드로 전환 & forward를 통해 출력 계산
    model.eval()
    with torch.no_grad():
        outputs_t0 = model.forward(s_batch_tensor, trace_batch_tensor, home_away_indicator_tensor)
    # 필요 시 numpy 배열로 변환 (TensorFlow의 sess.run()과 동일한 역할)
    readout_t1_batch = outputs_t0.numpy()

    return readout_t1_batch, home_away_indicator

In [15]:
# Q_predict
DATA_STORE = "../datastore"
MODEL_VERSION = 1920
DIR_GAMES_ALL = os.listdir(DATA_STORE)

for GAME_ID in tqdm(DIR_GAMES_ALL, desc="Q value 예측 및 저장"): 
    output, home_away_indicator = game_predict(FEATURE_NUMBER, hidden_dim, MAX_TRACE_LENGTH, learning_rate, SAVED_NETWORK, SPORT, MODEL_VERSION, GAME_ID)
    model_pre_dir = model_data_store_dir + f'/{GAME_ID}/'
    if not os.path.exists(model_pre_dir): 
        os.makedirs(model_pre_dir)
    savemat(model_pre_dir + f'{data_name}.mat', {'output': np.array(output, dtype='object')})
    savemat(model_pre_dir + f'home_identifier.mat', {'home_away_indicator': np.array(home_away_indicator, dtype='object')})


Q value 예측 및 저장: 100%|██████████| 64/64 [05:07<00:00,  4.81s/it]


### 3. GIM Calculation

[3-1] Episode(goal 기준)별로 Q value가 계산되어야 한다. <br>
[3-2] Episode의 첫 단계에선 계산하지 않는다. <br>
[3-3] Game별로 구하고, 모든 경기에 대해 선수별 GIM을 합산한다.

In [19]:
from player_ranking_match_by_match_td_lstm import GoalImpactMetric

In [63]:
PLAYER_ID_DICT_ALL_BY_MATCH = {}
PLAYER_ID_DICT_ALL = {}
DIFFERENCE_TYPE = "front_difference_"

# game 별로 계산 진행
for calibration_dir_game in tqdm(os.listdir(model_data_store_dir), desc="Q value calculating : .."):

    # model predict 결과물 | home identifier | playerId | training data | reward 모두 업로드
    for file_name in os.listdir(model_data_store_dir + "/" + calibration_dir_game):
        if file_name == data_name + ".mat":
            model_data_name = model_data_store_dir + "/" + calibration_dir_game + "/" + file_name
            model_data = (sio.loadmat(model_data_name))["output"]
        elif file_name.startswith("home_identifier"):
            home_identifier_name = model_data_store_dir + "/" + calibration_dir_game + "/" + file_name
            home_identifier = (sio.loadmat(home_identifier_name))["home_away_indicator"][0]
        else:
            continue
    playerIds_name = player_game_info_dir + "/" + calibration_dir_game + "/" + "playerId"
    playerIds = (sio.loadmat(playerIds_name))["playerId"][0]
    training_data_dict_all_name = "../datastore" + "/" + calibration_dir_game + "/" + "gamestates"
    training_data_dict_all = pd.read_csv(training_data_dict_all_name + ".csv")
    reward_name = "../datastore" + "/" + calibration_dir_game + "/" + "rnn_reward"
    reward = (sio.loadmat(reward_name))['data'][0]
        
    # player Q value 업데이트
    for player_Index in range(0, len(playerIds)):
        playerId = playerIds[player_Index]
        model_value = model_data[player_Index]
        # State_t와 State_t-1의 model 값 가져오기 (game의 시작과 끝부분 처리를 위해 조정)
        if player_Index - 1 >= 0:
            reward_pre = reward[player_Index - 1][0][-1]
            if reward_pre == 1:
                model_value_pre = model_data[player_Index]
            else:
                model_value_pre = model_data[player_Index - 1]
        else:
            model_value_pre = model_data[player_Index]
        if player_Index + 1 < len(playerIds):
            reward_pre = reward[player_Index - 1][0][-1]
            if reward_pre == 1:
                model_value_nex = model_data[player_Index]
            else:
                model_value_nex = model_data[player_Index + 1]
        else:
            model_value_nex = model_data[player_Index]
    
        if model_value[2][0] < 0:
            model_value[2][0] = 0
        if model_value_pre[2][0] < 0:
            model_value_pre[2][0] = 0
        if model_value_nex[2][0] < 0:
            model_value_nex[2][0] = 0
        
        ishome = home_identifier[player_Index][0]
        player_value = PLAYER_ID_DICT_ALL.get(playerId)

        # Q value 계산
        if player_value is None:
            if ishome:
                home_model_value = model_value[0][0]
                away_model_value = model_value[1][0]
                end_model_value = abs(model_value[2][0])
                home_model_value_pre = model_value_pre[0][0]
                away_model_value_pre = model_value_pre[1][0]
                end_model_value_pre = abs(model_value_pre[2][0])
                home_model_value_nex = model_value_nex[0][0]
                away_model_value_nex = model_value_nex[1][0]
                end_model_value_nex = abs(model_value_nex[2][0])
                # back_difference만 진행
                if DIFFERENCE_TYPE == "back_difference_":
                    q_value = (home_model_value - home_model_value_pre) - (
                            away_model_value - away_model_value_pre)
                elif DIFFERENCE_TYPE == "front_difference_":
                    q_value = (home_model_value_nex - home_model_value) - (
                            away_model_value_nex - away_model_value)
                elif DIFFERENCE_TYPE == "skip_difference_":
                    q_value = (home_model_value_nex - home_model_value_pre) - (
                            away_model_value_nex - away_model_value_pre)
                PLAYER_ID_DICT_ALL.update({playerId: {"value all": q_value}})

            else:
                home_model_value = model_value[0][0]
                away_model_value = model_value[1][0]
                end_model_value = abs(model_value[2][0])
                home_model_value_pre = model_value_pre[0][0]
                away_model_value_pre = model_value_pre[1][0]
                end_model_value_pre = abs(model_value_pre[2][0])
                home_model_value_nex = model_value_nex[0][0]
                away_model_value_nex = model_value_nex[1][0]
                end_model_value_nex = abs(model_value_nex[2][0])
                if DIFFERENCE_TYPE == "back_difference_":
                    q_value = (away_model_value - away_model_value_pre) - (
                            home_model_value - home_model_value_pre)
                elif DIFFERENCE_TYPE == "front_difference_":
                    q_value = (away_model_value_nex - away_model_value) - (
                            home_model_value_nex - home_model_value)
                elif DIFFERENCE_TYPE == "skip_difference_":
                    q_value = (away_model_value_nex - away_model_value_pre) - (
                            home_model_value_nex - home_model_value_pre
                    )
                PLAYER_ID_DICT_ALL.update(
                    {playerId: {
                        "value all": q_value}})
        else:
            if ishome:
                home_model_value = model_value[0][0]
                away_model_value = model_value[1][0]
                end_model_value = abs(model_value[2][0])
                home_model_value_pre = model_value_pre[0][0]
                away_model_value_pre = model_value_pre[1][0]
                end_model_value_pre = abs(model_value_pre[2][0])
                home_model_value_nex = model_value_nex[0][0]
                away_model_value_nex = model_value_nex[1][0]
                end_model_value_nex = abs(model_value_nex[2][0])
                if DIFFERENCE_TYPE == "back_difference_":
                    q_value = (home_model_value - home_model_value_pre) - (
                            away_model_value - away_model_value_pre)
                elif DIFFERENCE_TYPE == "front_difference_":
                    q_value = (home_model_value_nex - home_model_value) - (
                            away_model_value_nex - away_model_value)
                elif DIFFERENCE_TYPE == "skip_difference_":
                    q_value = (home_model_value_nex - home_model_value_pre) - (
                            away_model_value_nex - away_model_value_pre)
                player_value_number = player_value.get("value all") + q_value
            else:
                home_model_value = model_value[0][0]
                away_model_value = model_value[1][0]
                end_model_value = abs(model_value[2][0])
                home_model_value_pre = model_value_pre[0][0]
                away_model_value_pre = model_value_pre[1][0]
                end_model_value_pre = abs(model_value_pre[2][0])
                home_model_value_nex = model_value_nex[0][0]
                away_model_value_nex = model_value_nex[1][0]
                end_model_value_nex = abs(model_value_nex[2][0])
                if DIFFERENCE_TYPE == "back_difference_":
                    q_value = (away_model_value - away_model_value_pre) - (
                            home_model_value - home_model_value_pre)
                elif DIFFERENCE_TYPE == "front_difference_":
                    q_value = (away_model_value_nex - away_model_value) - (
                            home_model_value_nex - home_model_value)
                elif DIFFERENCE_TYPE == "skip_difference_":
                    q_value = (away_model_value_nex - away_model_value_pre) - (
                            home_model_value_nex - home_model_value_pre
                    )
                player_value_number = player_value.get("value all") + q_value
                
            PLAYER_ID_DICT_ALL.update(
                {playerId: {"value all": player_value_number}})


Q value calculating : ..: 100%|██████████| 64/64 [00:07<00:00,  8.66it/s]


In [65]:
len(PLAYER_ID_DICT_ALL)

602

In [74]:
df = pd.DataFrame.from_dict(PLAYER_ID_DICT_ALL, orient='index').reset_index()
df.rename(columns={'index': 'player_id'}, inplace=True)

# 'value all' 컬럼의 array에서 숫자만 꺼내는 작업
df['GIM'] = df['value all'].apply(lambda x: x[0])

# 결과 출력
df = df[['player_id', 'GIM']].copy()
df.head()

,player_id,GIM
0,5196,0.025991
1,5173,1.462126
2,5175,-0.106140
3,5178,0.892080
4,5187,0.017836


In [81]:
# CSV 파일 불러오기
player_database_df = pd.read_csv('../player_database.csv', index_col=0)

# 데이터프레임 병합 (player_id를 기준으로 병합)
player_df = pd.merge(player_database_df, df, on='player_id', how='outer')

# 결과 출력
print(player_df)

     player_id                      player_name  goal  shot       GIM
0       2941.0                     Ismaïla Sarr   0.0   3.0  0.265758
1       2948.0                      Nabil Fekir   0.0   3.0  0.640513
2       2954.0                  Youri Tielemans   0.0   4.0  0.282152
3       2974.0               Saîf-Eddine Khaoui   0.0   2.0 -0.020232
4       2995.0  Ángel Fabián Di María Hernández   1.0   4.0  0.690793
..         ...                              ...   ...   ...       ...
597    19597.0              Marcos Javier Acuña   0.0   1.0 -0.067859
598    19988.0                     Jimmy Durmaz   0.0   0.0  0.292513
599    20004.0                       Paul Pogba   1.0   5.0  2.813443
600    20005.0                Toby Alderweireld   0.0   5.0  0.953479
601    20016.0     Kléper Laveran Lima Ferreira   1.0   3.0  0.611182

[602 rows x 5 columns]


### 4. GIM statistics

In [84]:
# 'GIM' 칼럼 기준으로 내림차순 정렬
player_df_sorted_desc = player_df.sort_values(by='GIM', ascending=False)

print("\nTop 10 :")
player_df_sorted_desc.head(10)


Top 10 :


,player_id,player_name,goal,shot,GIM
91,3612.0,Ashley Young,0.0,0.0,4.520698
265,5487.0,Antoine Griezmann,4.0,15.0,4.224216
92,3621.0,Eden Hazard,3.0,16.0,4.025823
71,3501.0,Philippe Coutinho Correia,2.0,23.0,3.987090
136,5170.0,Mário Figueira Fernandes,1.0,1.0,3.845903
16,3083.0,Heung-Min Son,2.0,13.0,3.569811
121,4319.0,Edinson Roberto Cavani Gómez,3.0,14.0,3.476986
352,5574.0,Toni Kroos,1.0,15.0,3.396426
241,5463.0,Luka Modrić,2.0,8.0,3.320042
209,5246.0,Luis Alberto Suárez Díaz,2.0,10.0,3.256149


In [85]:
# 'goal' 칼럼 기준으로 내림차순 정렬
player_df_sorted_desc = player_df.sort_values(by='goal', ascending=False)

print("\nTop 10 :")
player_df_sorted_desc.head(10)


Top 10 :


,player_id,player_name,goal,shot,GIM
593,10955.0,Harry Kane,6.0,11.0,1.907858
172,5207.0,Cristiano Ronaldo dos Santos Aveiro,4.0,14.0,1.786480
152,5186.0,Denis Cheryshev,4.0,13.0,2.010219
44,3289.0,Romelu Lukaku Menama,4.0,14.0,2.601864
6,3009.0,Kylian Mbappé Lottin,4.0,8.0,1.831265
265,5487.0,Antoine Griezmann,4.0,15.0,4.224216
252,5474.0,Ivan Perišić,3.0,22.0,2.884089
92,3621.0,Eden Hazard,3.0,16.0,4.025823
525,6196.0,Yerry Fernando Mina González,3.0,3.0,1.349761
161,5195.0,Artem Dzyuba,3.0,7.0,2.320115
